# Split-Delivery Vehicle Routing Problem (Algorithm)

## Data

In [544]:
!rm -rf ds/*
!cp -r datasets/SetAInstance001.orders ds/
!cp -r datasets/SetAInstance001.network ds/

In [545]:
ORDER_DATA_FILE = 'ds/SetAInstance001.orders'
NETWORK_DATA_FILE = 'ds/SetAInstance001.network'

In [546]:
# Restructure order data
with open(ORDER_DATA_FILE, 'r') as fin:
    data = fin.read().splitlines(True)
    ds: list = list()
    for l in data:
        ds.append(l.replace('#', ''))

with open(ORDER_DATA_FILE, 'w') as fout:
    fout.writelines(ds[1:])
    
# Restructure network data
with open(NETWORK_DATA_FILE, 'r') as fin:
    data = fin.read().splitlines(True)
    ds: list = list()
    for l in data:
        ds.append(l.replace('#', ''))
with open(NETWORK_DATA_FILE, 'w') as fout:
    fout.writelines(ds[1:])
    
splitted_data: list = list()

with open(NETWORK_DATA_FILE, 'r') as fin:
    data = fin.read().splitlines(True)
    ds: list = list()
    reached_distance_set: bool = False
    
    for l in data:
        if 'network information' in l:
            reached_distance_set = True
            continue
        if reached_distance_set:
            splitted_data.append(l)
            
with open(NETWORK_DATA_FILE, 'w') as fout:
    fout.writelines(splitted_data)

In [547]:
import pandas as pd

In [548]:
order_data: pd.DataFrame = pd.read_csv(ORDER_DATA_FILE, sep=';', parse_dates=True)
network_data: pd.DataFrame = pd.read_csv(NETWORK_DATA_FILE, sep=';', parse_dates=True)

In [549]:
order_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ORD-keyword       500 non-null    object 
 1   number            500 non-null    int64  
 2   originIndex       500 non-null    int64  
 3   originCode        500 non-null    object 
 4   destinationIndex  500 non-null    int64  
 5   destinationCode   500 non-null    object 
 6   arrivalDate       500 non-null    object 
 7   dueDate           500 non-null    object 
 8   Unnamed: 8        0 non-null      float64
dtypes: float64(1), int64(3), object(5)
memory usage: 35.3+ KB


In [550]:
order_data.tail(10)

,ORD-keyword,number,originIndex,originCode,destinationIndex,destinationCode,arrivalDate,dueDate,Unnamed: 8
490,ORD,490,10,Wuppertal,13,Neuss,10.05.2022,11.05.2022,NaN
491,ORD,491,10,Wuppertal,13,Neuss,10.05.2022,12.05.2022,NaN
492,ORD,492,10,Wuppertal,13,Neuss,10.05.2022,11.05.2022,NaN
493,ORD,493,10,Wuppertal,7,Oberhausen,10.05.2022,12.05.2022,NaN
494,ORD,494,10,Wuppertal,7,Oberhausen,10.05.2022,11.05.2022,NaN
495,ORD,495,10,Wuppertal,0,Osnabrück,10.05.2022,12.05.2022,NaN
496,ORD,496,10,Wuppertal,27,Recklinghausen,10.05.2022,12.05.2022,NaN
497,ORD,497,10,Wuppertal,9,Solingen,10.05.2022,12.05.2022,NaN
498,ORD,498,10,Wuppertal,14,Viersen,10.05.2022,12.05.2022,NaN
499,ORD,499,10,Wuppertal,36,Witten,10.05.2022,11.05.2022,NaN


In [551]:
network_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1764 entries, 0 to 1763
Data columns (total 6 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   ODP-keyword(Origin-Destination-Pair)  1764 non-null   object
 1   originIndex                           1764 non-null   int64 
 2   originName                            1764 non-null   object
 3   destinationIndex                      1764 non-null   int64 
 4   destinationName                       1764 non-null   object
 5   distanceKM                            1764 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 82.8+ KB


In [552]:
network_data.head(25)

,ODP-keyword(Origin-Destination-Pair),originIndex,originName,destinationIndex,destinationName,distanceKM
0,ODP,0,Osnabrück,0,Osnabrück,0
1,ODP,0,Osnabrück,1,Düsseldorf,182
2,ODP,0,Osnabrück,2,Duisburg,161
3,ODP,0,Osnabrück,3,Essen,144
4,ODP,0,Osnabrück,4,Krefeld,183
5,ODP,0,Osnabrück,5,Mönchengladbach,205
6,ODP,0,Osnabrück,6,Mülheim an der Ruhr,154
7,ODP,0,Osnabrück,7,Oberhausen,152
8,ODP,0,Osnabrück,8,Remscheid,169
9,ODP,0,Osnabrück,9,Solingen,175


## Inspecting the dataset

In [553]:
order_data['arrivalDate'].min()

'01.05.2022'

In [554]:
order_data['arrivalDate'].max()

'10.05.2022'

In [555]:
order_data['dueDate'].min()

'02.05.2022'

In [556]:
order_data['dueDate'].max()

'12.05.2022'

# Order Allocation Algorithm (OAA)

## Grading System 

A grading system allows to score all orders based on fixed criteria. Afterwards, for each due date, the orders can be batched up to 6 cars per transport vehicle. For the grading system to be efficient, the criteria must optimize the cost of transport routes by minimizing the length while maximizing the amount of delivered cars per transporter.

In [557]:
from datetime import datetime, timedelta

def calculate_datetimes(today: str) -> (datetime, datetime, datetime):
    # Calculate datetimes for today (t), tomorrow (t+1), TDAD (t+2)
    today_datetime = datetime.strptime(today, '%d.%m.%Y')
    tomorrow_datetime = today_datetime + timedelta(days=1)
    tdat_datetime = tomorrow_datetime + timedelta(days=1)
    
    return (
        today_datetime,
        tomorrow_datetime,
        tdat_datetime,
    )

def grade_orders(
    orders: pd.DataFrame = order_data,
    network: pd.DataFrame = network_data,
    today: str = None,
) -> (pd.DataFrame, pd.DataFrame, dict[int, int]):
    if not today:
        today = orders['arrivalDate'].min()
    
    # Datetimes for today (t), tomorrow (t+1), TDAD (t+2)
    today_datetime, tomorrow_datetime, tdat_datetime = calculate_datetimes(today)
    
    # Reset scores for any previous grading.
    # Initially, every orders receives a score of 100 points.
    orders['score'] = 1
    
    # For every order, divide the score S by the number of days D until due date.
    # If only D = 1 day remains until due date, the score S is redeclared with S/D = S/1, leaving the score as it is.
    # If D > 1 day remains until due date, the score S is redeclared with S / D, decreasing the score.
    for index, row in orders.iterrows():
        arrival_datetime = datetime.strptime(row['arrivalDate'], '%d.%m.%Y')
        due_datetime = datetime.strptime(row['dueDate'], '%d.%m.%Y')
        days_delta = (due_datetime - arrival_datetime).days
        # Adjust score based on days delta
        orders.at[index, 'score'] = row['score'] / days_delta
    
    """
    # For every order, multiply the score S with the distance from the center.
    for index, row in orders.iterrows():
        # Get origin and destination index from order
        origin_index = row['originIndex']
        destination_index = row['destinationIndex']
        # Get distance from the origin (center) to the destination (dealer)
        distances = network.query(f"originIndex == {origin_index} and destinationIndex == {destination_index}")
        assert len(distances) == 1
        distance: int = distances['distanceKM']
        # Adjust score based on distance from center to destination
        orders.at[index, 'score'] = row['score'] * distance
    """
    
    # Calculating the mean distance between locations
    # TODO: calculating the mean without the distances from the center to the dealers
    mean_distance: float = network['distanceKM'].mean()
    print('LOG:', 'Mean distance:', mean_distance)
    
    # Compare every two orders with each other and determine whether their distance is less than the mean distance.
    # If their distance is less than the mean, they have a stronger metric relationship compared to other pairs.
    # If so, increasing the score will make both of them rank higher in the grading, which increases their possibility of being shipped together.
    for index, row in network.iterrows():
        origin_index = row['originIndex']
        destination_index = row['destinationIndex']
        distance = row['distanceKM']
        
        N = 5
        for n in range(1, N + 1):
            if distance <= (mean_distance / n):
                # - increase scores for origin, destination
                X = orders.query(f"destinationIndex == {destination_index}")
                Y = orders.query(f"destinationIndex == {origin_index}")
                
                for S in [X, Y]:
                    for i, r in X.iterrows():
                        orders.at[i, 'score'] = r['score'] * n
            else:
                break
    
    batches: dict[int, int] = {}
    
    td: pd.DataFrame = orders.query(f"arrivalDate == '{today_datetime.strftime('%d.%m.%Y')}' and dueDate == '{tomorrow_datetime.strftime('%d.%m.%Y')}'")
    ttd: pd.DataFrame = orders.query(f"arrivalDate == '{today_datetime.strftime('%d.%m.%Y')}' and dueDate == '{tdat_datetime.strftime('%d.%m.%Y')}'")
    
    deliveries: pd.DataFrame = pd.concat([td, ttd])
    
    # TODO: sort deliveries by descending score
    deliveries = deliveries.sort_values(by='score', ascending=False)
    
    # Create a mapping between a dealer ID and the number of ordered cars that can or need to be delivered today.
    for index, row in deliveries.iterrows():
        destination_index = row['destinationIndex']
        
        if destination_index not in batches.keys():
            batches[destination_index] = 1
        else:
            batches[destination_index] = batches[destination_index] + 1
        
        # Drop out row to enable data consistency.
        # The manipulated dataset should be saved after the algorithm was executed.
        orders.drop(axis=0, index=index)
    
    return (orders, network, batches)

In [558]:
%time
orders, network, batches = grade_orders()

CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 3.1 µs
LOG: Mean distance: 88.43197278911565


In [559]:
batches

{3: 1,
 22: 1,
 6: 1,
 31: 3,
 12: 1,
 35: 1,
 7: 3,
 2: 3,
 11: 2,
 10: 2,
 32: 3,
 1: 5,
 9: 2,
 33: 1,
 21: 1,
 17: 5,
 5: 3,
 37: 1,
 28: 2,
 34: 1,
 16: 1,
 30: 2,
 40: 2,
 24: 3}

In [560]:
# Test for the sum of cars that are being delivered today to make sure that
# the number of outgoing cars matches the number of incoming provided ones.
S = 0
for k, v in batches.items():
    S = S + v
S

50

In [561]:
def allocate(batches: dict[int, int]) -> list[list[int]]:
    allocations: list[list[int]] = []
    
    # Iterate over batched orderse to check whether the amount of vehicles exceeds >= 6.
    for key, value in batches.items():
        if value >= 6:
            X = value
            while X >= 6:
                allocations.append([key for k in range(0, 6)])
                X = X - 6
            batches[key] = X
    
    # Group remaining orders to allocations.
    # Iterate over the batches in a chronological order, with ensures
    # that the orders are allocated in the hierarchy of the scoring.
    for k_x, v_x in batches.items():
        allocation: list[int] = list()
        leftover: int = 6 - v_x
        
        for i in range(0, v_x):
            allocation.append(k_x)
        
        assert len(allocation) == v_x
        
        batches[k_x] = 0
        
        for k_y, v_y in batches.items():
            if k_x != k_y:
                if v_y == leftover:
                    for i in range(0, v_y):
                        allocation.append(k_y)
                    batches[k_y] = 0
                    break
                else:
                    continue
        
        allocations.append(allocation)
    
    # Fill available slots in the allocations.
    for A in allocations:
        L = len(A)
        if L < 6:
            allocation: list[int] = list()
            allocations.remove(A)
            allocation = [*A]
            slots: int = 6 - L
            for B in allocations:
                if len(B) == slots:
                    allocations.remove(B)
                    allocation = [*allocation, *B]
                    break
                elif len(B) < slots:
                    allocations.remove(B)
                    allocation = [*allocation, *B]
                    slots = slots - len(B)
                    
            allocations.append(allocation)
            
    
    assert len(allocations) <= 10
    
    return allocations

In [562]:
%time
allocations: list[list[int]] = allocate(batches)
allocations

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.1 µs


[[3, 1, 1, 1, 1, 1],
 [22, 17, 17, 17, 17, 17],
 [31, 31, 31, 7, 7, 7],
 [2, 2, 2, 32, 32, 32],
 [5, 5, 5, 24, 24, 24],
 [6, 12, 11, 11, 9, 9],
 [35, 10, 10, 33, 37, 34],
 [16, 40, 40],
 [21, 28, 28, 30, 30]]

In [563]:
S = 0
for A in allocations:
    S = S + len(A)
S

50

## Route Optimization

In [564]:
from typing import Union

def distance(
    originIndex: int,
    destinationIndex: int,
    network: pd.DataFrame = network,
) -> Union[int, None]:
    for index, row in network.iterrows():
        if row['originIndex'] == originIndex and row['destinationIndex'] == destinationIndex:
            return row['distanceKM']

In [565]:
from typing import Union

def query_station_index(
    station_name: str,
    network: pd.DataFrame = network,
) -> Union[int, None]:
    for index, row in network.iterrows():
        if row['originName'] == station_name:
            return row['originIndex']

In [566]:
def find_next(i, l):
    # find the shortest route betwteen dealer i and others
    x = len(l) - 1
    dis_tmp = distance(i, l[x])
    ct = l[x]
    for n in range(x):
        if dis_tmp > distance(i, l[n]):
            dis_tmp = distance(i, l[n])
            ct = l[n]
    return ct

def direction(route):
    city_forwards = 0
    cars_forwards = len(route) - 2
    city_backwards = len(route) - 2
    cars_backwards = cars_forwards
    go_forward = 0
    go_reverse = 0
    l = len(route)
    for i in range(l - 1):
        go_forward += distance(route[city_forwards], route[city_forwards + 1]) * cars_forwards
        cars_forwards -= 1
        city_forwards += 1
    for i in range(l - 1):
        go_reverse += distance(route[city_backwards], route[city_backwards - 1]) * cars_backwards
        cars_backwards -= 1
        city_backwards -= 1
    if go_reverse < go_forward:
        route.reverse()
    return route

def find_route(
    l: list[int],
    station_name: str = 'Wuppertal',
    network: pd.DataFrame = network,
):
    station_index: int = query_station_index(station_name=station_name, network=network)
    assert station_index != None
    
    c_l: list[int] = list()
    for i in range(len(l)):
        c_l.append(l[i])
    length = len(c_l)
    route: list[int] = list()
    route.append(station_index)
    for m in range(length):
        l = len(route)
        x = find_next(route[l - 1], c_l)
        route.append(x)
        c_l.remove(x)
    route.append(station_index)
    direction(route)
    return route

In [567]:
def optimize_allocations(allocations: list[list[int]]) -> list[list[int]]:
    optimized_allocations: list[list[int]] = list()
    
    for allocation in allocations:
        A: list[int] = find_route(allocation)
        optimized_allocations.append(A)
    
    return optimized_allocations

In [568]:
optimized_allocations: list[list[int]] = optimize_allocations(allocations)
optimized_allocations

[[10, 1, 1, 1, 1, 1, 3, 10],
 [10, 17, 17, 17, 17, 17, 22, 10],
 [10, 7, 7, 7, 31, 31, 31, 10],
 [10, 2, 2, 2, 32, 32, 32, 10],
 [10, 24, 24, 24, 5, 5, 5, 10],
 [10, 9, 9, 11, 11, 6, 12, 10],
 [10, 10, 10, 33, 37, 34, 35, 10],
 [10, 40, 40, 16, 10],
 [10, 28, 28, 30, 30, 21, 10]]

In [569]:
S = 0
for A in optimized_allocations:
    S = S + len(A)
S

68

In [570]:
def calculate_cumulative_distance(allocations: list[list[int]]) -> int:
    S: int = 0
    
    for A in allocations:
        for i in range(1, len(A)):
            d: int = distance(A[i-1], A[i])
            S = S + d

    return S

In [571]:
total_distance: int = calculate_cumulative_distance(optimized_allocations)
total_distance

1918

In [572]:
average_distance = total_distance / (S - (len(optimized_allocations) * 2))
average_distance

38.36